In [4]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable

In [5]:
# Create the LP problem
prob = LpProblem("Maximize_Profit", LpMaximize)

# Define decision variables
x1 = LpVariable("x1", lowBound=0)
x2 = LpVariable("x2", lowBound=0)

# Define the DataFrame with coefficients and bounds
df_constraints = pd.DataFrame(
    {
        "x1_coeff": [2, 1],  # Coefficients for x1
        "x2_coeff": [1, 3],
        # Coefficients for x2
        "lower_bound": [10, 15],  # Lower bounds for the linear combinations
        "upper_bound": [20, 25],  # Upper bounds for the linear combinations
    }
)

# Objective function
prob += 3 * x1 + 4 * x2  # Example objective function

# Iterate through the DataFrame and add constraints
for i, row in df_constraints.iterrows():
    # Add the lower bound constraint: row['x1_coeff']*x1 + row['x2_coeff']*x2 >= row['lower_bound']
    prob += row["x1_coeff"] * x1 + row["x2_coeff"] * x2 >= row["lower_bound"]

    # Add the upper bound constraint: row['x1_coeff']*x1 + row['x2_coeff']*x2 <= row['upper_bound']
    prob += row["x1_coeff"] * x1 + row["x2_coeff"] * x2 <= row["upper_bound"]

# Solve the problem
prob.solve()

# Results
print(f"Optimal solution: x1 = {x1.varValue}, x2 = {x2.varValue}")
print(f"Optimal value: {prob.objective.value()}")

Optimal solution: x1 = 7.0, x2 = 6.0
Optimal value: 45.0


In [102]:
cnap = pd.read_csv('core_nutrient_amounts_prices_v3.csv', index_col=0)
cnap = cnap.rename(columns={'name': 'nutrient_name'})
constraints = pd.read_csv('nutrient_constraints_csv.csv').set_index('nutrient_nbr').drop(columns=['id', 'rank']).rename(columns={'name': 'nutrient_name'})
constraints = constraints.loc[constraints.target.notna() | constraints.ll.notna() | constraints.ul.notna()]    
n_values = cnap.pivot(index='nutrient_nbr', columns='food', values='nutrient_unit_per_dollar')
cc = constraints.join(n_values, how='left')

In [127]:
prob = LpProblem("Minimize_Cost", LpMaximize)
decision_variables = []
first_food_idx = cc.columns.get_loc('Almonds')
for food in cc.columns[first_food_idx:]:
    decision_variables.append(LpVariable(f"{food}", lowBound=0))
for i, row in cc.iterrows():
    constraint = 0
    for j, food in enumerate(cc.columns[first_food_idx:]):
        constraint += decision_variables[j] * row[food]
    if pd.notna(row.target):
        prob += constraint == row.target
    elif pd.notna(row.ll):
        prob += constraint >= row.ll
    elif pd.notna(row.ul):
        prob += constraint <= row.ul

In [144]:
prob.solve()
for v in prob.variables():
    if v.varValue != 0:
        print(v.name, "=", v.varValue)

Almonds = 0.56273633
Beef_Liver = 0.03739722
Carrots = 2.4840002
Grapefruit = 2.5735587
Lentils = 0.12942993
Oysters = 3.003217
Salmon = 7.2746342
__dummy = None


In [166]:
fs = pd.DataFrame([(v.name, v.varValue) for v in prob.variables() if v.varValue != 0], columns=['food', 'spend']).iloc[:-1,:]
cn = cc.set_index('nutrient_name').iloc[:, 4:]
# Rank the foods for each nutrient
food_ranks = cn.rank(axis=1, method='min', ascending=False)
final_foods = cnap.loc[cnap.food.isin([v.name for v in prob.variables() if v.varValue != 0])].drop_duplicates('food')

In [169]:
ffs = final_foods.merge(fs, on='food')

In [171]:
ffs.spend / ffs.price_per_100_g

0    14.538356
1     9.760415
2     0.638132
3     4.124645
4     1.362235
5     0.435038
dtype: float64

In [176]:
ffs.spend.sum() * 365

5850.0653714

In [173]:
14 * 100 / 453.592

3.086474188257289

In [174]:
# TODO: add fiber constraint, maybe variety constraint, check numbers so that oysters probably don't show up